In [12]:
import cv2
import numpy as np
import os
import pafy
import math
import random 
import datetime
import tensorflow as tf 
from collections import deque
import matplotlib.pyplot as plt 

from moviepy.editor import *

from sklearn.model_selection import train_test_split

from tensorflow.keras.layers import *
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical, plot_model
from tensorflow.keras.callbacks import EarlyStopping



In [ ]:
https://www.youtube.com/watch?v=QmtSkq3DYko  

In [18]:
seed_constant =  27
np.random.seed(seed_constant)
random.seed(seed_constant)
tf.random.set_seed(seed_constant)

In [14]:
# Resize the video frames 
Im_height, Im_width = 64, 64 

# the number of frames fed to the model in one sequence
sequence_length = 20

dataset_dir = 'UCF50'

# train on the following classes 
classes_list = ["WalkingWithDog","TaiChi","Swing","HorseRace"]

In [4]:
def frame_exctraction(video_path):
    '''
    This function will extract the required frames from a video after resizing and normalizing them.

    Args:
        video_path : The path of the video in the disk, whose frames are to be exctracted 
    Returns:
        frame_list : A list containing the resized and normalized frames of the video

    '''

    frame_list = []

    video_reader = cv2.VideoCapture(video_path)

    video_frames_count = int(video_reader.get(cv2.CAP_PROP_FRAME_COUNT))

    frame_interval = max(int(video_frames_count/sequence_length),1)

    for frame_counter in range(sequence_length):
        video_reader.set(cv2.CAP_PROP_POS_FRAMES, frame_counter * frame_interval)

        success, frame = video_reader.read()

        if not success:
            break

        resized_frame = cv2.resize(frame,(Im_height,Im_width))
 
        normalized_frame = resized_frame / 255

        frame_list.append(normalized_frame)

    video_reader.release()

    return frame_list 



In [6]:
def create_dataset():
    '''
    This function will exctract the data of the selected classes and create the required dataset
    Returns: 
        features:  A list containing the extracted frames of the videos. 
        labels:    A list containing the indexes of the classes
    '''

    features = []
    labels = []
    video_files_paths = []

    for class_index, class_name in enumerate(classes_list):
        print(f"Exctracting Data of class: {class_name}")

        files_list = os.listdir(os.path.join(dataset_dir, class_name))

        for file_name in files_list:

            video_file_path = os.path.join(dataset_dir, class_name, file_name)

            frames =  frame_exctraction(video_file_path)

            if len(frames) == sequence_length:

                features.append(frames) 
                labels.append(class_index)
                video_files_paths.append(video_file_path)

    features = np.asarray(features)
    labels = np.array(labels)

    return features, labels, video_files_paths

In [15]:
# create the dataset
features, labels, video_files_paths = create_dataset()

Exctracting Data of class: WalkingWithDog
Exctracting Data of class: TaiChi
Exctracting Data of class: Swing
Exctracting Data of class: HorseRace


In [16]:
# labels into one-hot encoded vectors
one_hot_encoded_labels = to_categorical(labels)

In [19]:
# split the dataset

features_train, features_test, labels_train, labels_test = train_test_split(features, one_hot_encoded_labels, test_size=0.25, shuffle =True, random_state=seed_constant)

In [20]:
#convLSTM
def create_convlstm():
    model =  Sequential()

    model.add(ConvLSTM2D(filters = 4,kernel_size=(3,3), activation='tanh', data_format="channels_last",recurrent_dropout=0.2,return_sequences=True, 
                        input_shape=(sequence_length,Im_height,Im_width,3)))
    model.add(TimeDistributed(Dropout(0.2)))


    model.add(ConvLSTM2D(filters = 8,kernel_size=(3,3), activation='tanh', data_format="channels_last",recurrent_dropout=0.2,return_sequences=True))


    model.add(MaxPooling3D(pool_size=(1,2,2), padding='same', data_format="channels_last"))
    model.add(TimeDistributed(Dropout(0.2)))


    model.add(ConvLSTM2D(filters = 14,kernel_size=(3,3), activation='tanh', data_format="channels_last",recurrent_dropout=0.2,return_sequences=True))

    model.add(MaxPooling3D(pool_size=(1,2,2), padding='same', data_format="channels_last"))
    model.add(TimeDistributed(Dropout(0.2)))

    model.add(ConvLSTM2D(filters = 16,kernel_size=(3,3), activation='tanh', data_format="channels_last",recurrent_dropout=0.2,return_sequences=True))
    model.add(MaxPooling3D(pool_size=(1,2,2), padding='same', data_format="channels_last"))

    model.add(Flatten())

    model.add(Dense(len(classes_list), activation="softmax"))

    model.summary()

    return model

In [21]:
convlstm_model = create_convlstm()


2024-01-28 03:20:05.123454: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-01-28 03:20:05.153623: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-01-28 03:20:05.153732: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

NameError: name 'channels_last' is not defined